In [ ]:
import re
import json
import pandas as pd

pd.set_option("max_colwidth", 200)

## Load data

In [ ]:
file_path = "/workspace/notebooks/dev/patterns/03-sections-spot/validacion_codigos.csv"

df = pd.read_csv(file_path)
df.info()

In [ ]:
df.rename(columns={column: column.lower() for column in df.columns}, inplace=True)

In [ ]:
df.sample(5)

## Materia

In [ ]:
df["materia"].nunique()

In [ ]:
df["materia"].value_counts()

In [ ]:
df.loc[df["materia"].isin(("allanamiento_autonomo", "habeas_corpus", "ejecucion_de_multa", "amparo"))]

In [ ]:
df.dropna(subset=["art_infringido"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

In [ ]:
df["materia"].nunique()

In [ ]:
df["materia"].value_counts()

## Código o Ley

In [ ]:
df["codigo_o_ley"].nunique()

In [ ]:
df["codigo_o_ley"].value_counts()

In [ ]:
pd.crosstab(df["materia"], df["codigo_o_ley"]).T

In [ ]:
subject = df.groupby("codigo_o_ley")["materia"].aggregate(set).map(list)
subject

In [ ]:
subject_map = subject.loc[subject.map(lambda x: len(x) == 1)].map(lambda x: x[0]).to_dict()
subject_map

## Artículo infringido

In [ ]:
df["art_infringido"].nunique()

In [ ]:
df["art_infringido"].sample(5)

In [ ]:
df["art_infringido"] = df["art_infringido"].str.replace("bis", " bis")
df["art_infringido"] = df["art_infringido"].str.replace("ter", " ter")
df["art_infringido"] = df["art_infringido"].map(lambda x: re.sub(r"_inc(\d+)", r" inc \1", x))
df["art_infringido"].nunique()

In [ ]:
df[df.duplicated(subset=["art_infringido"], keep=False)].sort_values(["art_infringido", "codigo_o_ley"])

In [ ]:
df[df.duplicated(subset=["art_infringido"])]["art_infringido"].nunique()

In [ ]:
df[df.duplicated(subset=["art_infringido", "codigo_o_ley"], keep=False)].sort_values(["art_infringido", "codigo_o_ley"])

In [ ]:
df[df.duplicated(subset=["art_infringido", "codigo_o_ley"])]["art_infringido"].nunique()

In [ ]:
arts = df.groupby("art_infringido")["codigo_o_ley"].aggregate(set).map(list)
arts

In [ ]:
arts.map(lambda x: len(x) == 1).sum() / len(arts)

In [ ]:
arts.loc[arts.map(lambda x: len(x) > 1)]

In [ ]:
df.loc[df["art_infringido"].isin(arts.loc[arts.map(lambda x: len(x) > 1)].index)].sort_values(["art_infringido", "codigo_o_ley"])

In [ ]:
direct_map = arts.loc[arts.map(lambda x: len(x) == 1)].map(lambda x: x[0]).to_dict()
direct_map

In [ ]:
len(direct_map.keys())

In [ ]:
df.loc[~df["art_infringido"].isin(direct_map.keys())].sort_values(["art_infringido", "codigo_o_ley"])

In [ ]:
indirect_map = df.loc[
    ~df["art_infringido"].isin(direct_map.keys()),
    ["art_infringido", "conducta", "codigo_o_ley"],
].sort_values(["art_infringido", "codigo_o_ley"])

indirect_map

In [ ]:
indirect_map_groups = indirect_map.groupby("art_infringido").groups
indirect_map_groups

In [ ]:
for art, idx in indirect_map_groups.items():
    indirect_map_groups.update({art: {indirect_map.loc[i]["conducta"]: indirect_map.loc[i]["codigo_o_ley"] for i in idx}})

In [ ]:
print(json.dumps(indirect_map_groups, indent=4))